In [7]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("C:\\Users\\astri\\class_work\\WeatherPy\\cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
City_ID,,,,,,,,,
0,adamstown,-25.0660,-130.1015,22.477778,72,31,6.89,PN,1712428394
1,port-aux-francais,-49.3500,70.2167,9.022222,94,100,34.67,TF,1712428394
2,makokou,0.5738,12.8642,22.000000,93,100,1.63,GA,1712428680
3,ushuaia,-54.8000,-68.3000,4.811111,65,75,32.21,AR,1712428680
4,margaret river,-33.9500,115.0667,15.300000,75,1,13.20,AU,1712428397


In [19]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE


def map_humidity_to_size(humidity):
  
    return humidity * 10


map_plot = city_data_df.hvplot.points('Lng', 'Lat', geo=True, tiles='CartoLight',
                                       title='City Locations with Humidity', 
                                       color='green', alpha=0.50, 
                                       size=map_humidity_to_size(city_data_df['Humidity']),
                                       hover_cols=['City', 'Humidity'])



final_plot = (map_plot).opts(width=1000, height=800)


final_plot







:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (_size,City,Humidity)

In [9]:


# Filtered cities with temperatures above 22 degrees Celsius
filtered_df = city_data_df.loc[city_data_df['Max Temp'] > 22]


filtered_df = filtered_df.loc[filtered_df.notna().all(axis=1)]


filtered_df = filtered_df.reset_index(drop=True)

filtered_df 


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,adamstown,-25.0660,-130.1015,22.477778,72,31,6.89,PN,1712428394
1,ta`u,-14.2336,-169.5144,28.450000,74,100,5.68,AS,1712428681
2,san julian,21.0167,-102.1667,26.600000,15,100,14.74,MX,1712428681
3,anna regina,7.2644,-58.5077,29.777778,62,96,8.01,GY,1712428681
4,vila velha,-20.3297,-40.2925,29.422222,83,20,12.66,BR,1712428565
...,...,...,...,...,...,...,...,...,...
255,rabaul,-4.1967,152.1721,27.400000,82,100,8.79,PG,1712428756
256,albertina,-22.2008,-46.6158,25.900000,57,53,6.13,BR,1712428757
257,salinopolis,-0.6136,-47.3561,28.777778,72,78,9.35,BR,1712428470
258,key west,24.5557,-81.7826,26.000000,55,20,8.05,US,1712428471


In [10]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity

hotel_df = filtered_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""
hotel_df 

,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,72,
1,ta`u,AS,-14.2336,-169.5144,74,
2,san julian,MX,21.0167,-102.1667,15,
3,anna regina,GY,7.2644,-58.5077,62,
4,vila velha,BR,-20.3297,-40.2925,83,
...,...,...,...,...,...,...
255,rabaul,PG,-4.1967,152.1721,82,
256,albertina,BR,-22.2008,-46.6158,57,
257,salinopolis,BR,-0.6136,-47.3561,72,
258,key west,US,24.5557,-81.7826,55,


In [16]:
# Set parameters to search for a hotel
radius = 10000
#categories = "accomodation.hotel"
#limit= 20


params = {"apiKey": geoapify_key, 
         "categories": "accommodation.hotel",
         "limit" : 20
             
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    # YOUR CODE HERE
    latitude = row['Lat']
    longitude = row['Lng']

    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =  requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address_response = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address_response["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df 

Starting hotel search
adamstown - nearest hotel: No hotel found
ta`u - nearest hotel: No hotel found
san julian - nearest hotel: No hotel found
anna regina - nearest hotel: Jaigobin Hotel
vila velha - nearest hotel: Hotel Prainha
urucui - nearest hotel: Formula Flat hotel
ca mau - nearest hotel: Khách sạn Song Ngọc
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
port mathurin - nearest hotel: Escale Vacances
isla aguada - nearest hotel: Puerto Sabalo Hotel
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
georgetown - nearest hotel: Page 63 hostel
barrouallie - nearest hotel: High Field Guesthouse
papao - nearest hotel: Hiti Moana Villa Lodge
minab - nearest hotel: هتل صدف
saipan - nearest hotel: Chalan Kanoa Beach Hotel
kisangani - nearest hotel: Hotel Kisangani
kolonia - nearest hotel: Wasserturm Hotel Cologne
nova sintra - nearest hotel: Residência Ka Dencho
utrik - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
hawaiian par

,City,Country,Lat,Lng,Humidity,Hotel Name
0,adamstown,PN,-25.0660,-130.1015,72,No hotel found
1,ta`u,AS,-14.2336,-169.5144,74,No hotel found
2,san julian,MX,21.0167,-102.1667,15,No hotel found
3,anna regina,GY,7.2644,-58.5077,62,Jaigobin Hotel
4,vila velha,BR,-20.3297,-40.2925,83,Hotel Prainha
...,...,...,...,...,...,...
255,rabaul,PG,-4.1967,152.1721,82,Rabaul Hotel
256,albertina,BR,-22.2008,-46.6158,57,Hotel Parque das Primaveras
257,salinopolis,BR,-0.6136,-47.3561,72,Hotel Salinópolis
258,key west,US,24.5557,-81.7826,55,Harborside Hotel & Marina


In [27]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

map_plot_hotel = hotel_df.hvplot.points('Lng', 'Lat', geo=True, tiles='CartoLight',
                                       title='City Locations', 
                                       color='green', alpha=0.50, 
                                       
                                       hover_cols=['City', 'Country', 'Hotel Name'])
# Display the map
# YOUR CODE HERE

map_plot_hotel 

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Country,Hotel Name)